# CMS Data Exercise 

## Author Infomation

Name : Darryn Johnson 

Date Last Updated : 2/2/2025

## Sources and Links

Original Dataset : [Payroll Based Journal Daily Nurse Staffing](https://data.cms.gov/quality-of-care/payroll-based-journal-daily-nurse-staffing/data)

# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Reproducability

# Business Understanding 

The purpose of the following notebook is to look into the data associated with staffing long-term care facilities. We will be using data such as `provider Name`, provider `City` and `State`, as well as other metrics used to identify 

# Exploritory Data Analysis

In [2]:
df = pd.read_csv('PBJ_Daily_Nurse_Staffing_Q2_2024.zip', encoding = 'Latin-1', low_memory = False)

In [3]:
health = pd.read_csv('NH_HealthCitations_Nov2024.csv', low_memory = False)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1325324 entries, 0 to 1325323
Data columns (total 33 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   PROVNUM           1325324 non-null  object 
 1   PROVNAME          1325324 non-null  object 
 2   CITY              1325324 non-null  object 
 3   STATE             1325324 non-null  object 
 4   COUNTY_NAME       1325324 non-null  object 
 5   COUNTY_FIPS       1325324 non-null  int64  
 6   CY_Qtr            1325324 non-null  object 
 7   WorkDate          1325324 non-null  int64  
 8   MDScensus         1325324 non-null  int64  
 9   Hrs_RNDON         1325324 non-null  float64
 10  Hrs_RNDON_emp     1325324 non-null  float64
 11  Hrs_RNDON_ctr     1325324 non-null  float64
 12  Hrs_RNadmin       1325324 non-null  float64
 13  Hrs_RNadmin_emp   1325324 non-null  float64
 14  Hrs_RNadmin_ctr   1325324 non-null  float64
 15  Hrs_RN            1325324 non-null  float64
 16  

In [5]:
df.head(3)

,PROVNUM,PROVNAME,CITY,STATE,COUNTY_NAME,COUNTY_FIPS,CY_Qtr,WorkDate,MDScensus,Hrs_RNDON,...,Hrs_LPN_ctr,Hrs_CNA,Hrs_CNA_emp,Hrs_CNA_ctr,Hrs_NAtrn,Hrs_NAtrn_emp,Hrs_NAtrn_ctr,Hrs_MedAide,Hrs_MedAide_emp,Hrs_MedAide_ctr
0,015009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240401,51,10.77,...,0.0,160.08,160.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,015009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240402,52,8.43,...,0.0,135.95,135.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,015009,"BURNS NURSING HOME, INC.",RUSSELLVILLE,AL,Franklin,59,2024Q2,20240403,53,11.13,...,0.0,150.31,150.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df.rename(columns = {'WorkDate' : 'Date'}, inplace = True)

In [7]:
df['Date'] = pd.to_datetime(df['Date'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1325324 entries, 0 to 1325323
Data columns (total 33 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   PROVNUM           1325324 non-null  object        
 1   PROVNAME          1325324 non-null  object        
 2   CITY              1325324 non-null  object        
 3   STATE             1325324 non-null  object        
 4   COUNTY_NAME       1325324 non-null  object        
 5   COUNTY_FIPS       1325324 non-null  int64         
 6   CY_Qtr            1325324 non-null  object        
 7   Date              1325324 non-null  datetime64[ns]
 8   MDScensus         1325324 non-null  int64         
 9   Hrs_RNDON         1325324 non-null  float64       
 10  Hrs_RNDON_emp     1325324 non-null  float64       
 11  Hrs_RNDON_ctr     1325324 non-null  float64       
 12  Hrs_RNadmin       1325324 non-null  float64       
 13  Hrs_RNadmin_emp   1325324 non-null  float6

In [9]:
df['CY_Qtr'].value_counts()

CY_Qtr
2024Q2    1325324
Name: count, dtype: int64

In [10]:
health.head(3)

,CMS Certification Number (CCN),Provider Name,Provider Address,City/Town,State,ZIP Code,Survey Date,Survey Type,Deficiency Prefix,Deficiency Category,...,Deficiency Corrected,Correction Date,Inspection Cycle,Standard Deficiency,Complaint Deficiency,Infection Control Inspection Deficiency,Citation under IDR,Citation under IIDR,Location,Processing Date
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2023-03-02,Health,F,Infection Control Deficiencies,...,"Deficient, Provider has date of correction",2023-04-06,1,Y,N,N,N,N,"701 MONROE STREET NW,RUSSELLVILLE,AL,35653",2024-11-01
1,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2023-03-02,Health,F,Resident Assessment and Care Planning Deficien...,...,Past Non-Compliance,2023-01-13,1,Y,Y,N,N,N,"701 MONROE STREET NW,RUSSELLVILLE,AL,35653",2024-11-01
2,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2023-03-02,Health,F,Quality of Life and Care Deficiencies,...,Past Non-Compliance,2023-01-13,1,Y,Y,N,N,N,"701 MONROE STREET NW,RUSSELLVILLE,AL,35653",2024-11-01


In [11]:
health.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404615 entries, 0 to 404614
Data columns (total 23 columns):
 #   Column                                   Non-Null Count   Dtype 
---  ------                                   --------------   ----- 
 0   CMS Certification Number (CCN)           404615 non-null  object
 1   Provider Name                            404615 non-null  object
 2   Provider Address                         404615 non-null  object
 3   City/Town                                404615 non-null  object
 4   State                                    404615 non-null  object
 5   ZIP Code                                 404615 non-null  int64 
 6   Survey Date                              404615 non-null  object
 7   Survey Type                              404615 non-null  object
 8   Deficiency Prefix                        404615 non-null  object
 9   Deficiency Category                      404615 non-null  object
 10  Deficiency Tag Number                    404

In [12]:
health.rename(columns = {'Survey Date' : 'Date'}, inplace = True)

In [13]:
health['Date'].value_counts()

Date
2024-01-25    1032
2023-11-16    1029
2024-06-13    1022
2024-01-11    1020
2024-05-23    1003
              ... 
2020-09-07       1
2020-11-11       1
2024-09-08       1
2022-11-24       1
2017-10-24       1
Name: count, Length: 2157, dtype: int64

In [14]:
health['Date'] = pd.to_datetime(health['Date'])

In [20]:
health_q = health[(health['Date'] >= '2024-4-1') & (health['Date'] <= '2024-6-30')].copy()

In [21]:
health_q['Date'] = pd.to_datetime(df['Date'])

In [16]:
health_q.head()

,CMS Certification Number (CCN),Provider Name,Provider Address,City/Town,State,ZIP Code,Date,Survey Type,Deficiency Prefix,Deficiency Category,...,Deficiency Corrected,Correction Date,Inspection Cycle,Standard Deficiency,Complaint Deficiency,Infection Control Inspection Deficiency,Citation under IDR,Citation under IIDR,Location,Processing Date
18,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2024-05-16,Health,F,"Freedom from Abuse, Neglect, and Exploitation ...",...,"Deficient, Provider has date of correction",2024-06-03,1,N,Y,N,N,N,"7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206",2024-11-01
189,015047,"NORTHWAY HEALTH AND REHABILITATION, LLC",1424 NORTH 25TH STREET,BIRMINGHAM,AL,35234,2024-04-17,Health,F,Resident Rights Deficiencies,...,"Deficient, Provider has date of correction",2024-05-22,1,N,Y,N,N,N,"1424 NORTH 25TH STREET,BIRMINGHAM,AL,35234",2024-11-01
190,015047,"NORTHWAY HEALTH AND REHABILITATION, LLC",1424 NORTH 25TH STREET,BIRMINGHAM,AL,35234,2024-04-17,Health,F,Resident Rights Deficiencies,...,"Deficient, Provider has date of correction",2024-05-22,1,Y,Y,N,N,N,"1424 NORTH 25TH STREET,BIRMINGHAM,AL,35234",2024-11-01
191,015047,"NORTHWAY HEALTH AND REHABILITATION, LLC",1424 NORTH 25TH STREET,BIRMINGHAM,AL,35234,2024-04-17,Health,F,Resident Rights Deficiencies,...,"Deficient, Provider has date of correction",2024-05-22,1,Y,Y,N,N,N,"1424 NORTH 25TH STREET,BIRMINGHAM,AL,35234",2024-11-01
192,015047,"NORTHWAY HEALTH AND REHABILITATION, LLC",1424 NORTH 25TH STREET,BIRMINGHAM,AL,35234,2024-04-17,Health,F,"Freedom from Abuse, Neglect, and Exploitation ...",...,"Deficient, Provider has date of correction",2024-05-22,1,Y,Y,N,N,N,"1424 NORTH 25TH STREET,BIRMINGHAM,AL,35234",2024-11-01


In [17]:
health_q.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30170 entries, 18 to 404614
Data columns (total 23 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   CMS Certification Number (CCN)           30170 non-null  object        
 1   Provider Name                            30170 non-null  object        
 2   Provider Address                         30170 non-null  object        
 3   City/Town                                30170 non-null  object        
 4   State                                    30170 non-null  object        
 5   ZIP Code                                 30170 non-null  int64         
 6   Date                                     30170 non-null  datetime64[ns]
 7   Survey Type                              30170 non-null  object        
 8   Deficiency Prefix                        30170 non-null  object        
 9   Deficiency Category                      3

In [22]:
df3 = df.join(health_q, how = 'left', on = 'Date')

ValueError: You are trying to merge on datetime64[ns] and int64 columns for key 'Date'. If you wish to proceed you should use pd.concat

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df3.head()

In [ ]:
df3['Inspection Cycle'].value_counts()